In [1]:
%run "Properati_base_setup.py"

Running command `conda list`... ok
pandas=1.0.3 already installed
matplotlib=2.2.2 already installed
bokeh=2.0.0 already installed
seaborn=0.10.0 already installed
ipywidgets=7.5.1 already installed
pytest=5.3.4 already installed
chardet=3.0.4 already installed
plotly=4.6.0 already installed
chart-studio=1.1.0 already installed
nodejs=13.13.0 already installed
psutil=5.7.0 already installed
plotly-orca=1.3.1 already installed


In [2]:
import re 
import numpy as np 
import pandas as pd

In [3]:
data = pd.read_csv("properati.csv", sep=",")

In [4]:
print("cantidad de filas: " + str(data.shape[0]))
print("cantidad de columnas: " + str(data.shape[1]))

cantidad de filas: 121220
cantidad de columnas: 26


In [5]:
pattern = r'http:\/\/www\.properati.com.ar\/\w{6}([\w*-]*)'
pattern_regex = re.compile(pattern)
result_match = data['properati_url'].apply(lambda x: pattern_regex.match(x.lower().replace('-','_')))
result =  result_match.apply(lambda x: x.group(1).split('_'))

list_words = []

def generate_words_series(arr):
    for x in arr:
        list_words.append(str(x))
    return

result.apply(generate_words_series)

serie = pd.Series(list_words)
key_words = serie.value_counts()

In [6]:
display(key_words.head(40))

venta            121119
departamento      71019
garage            56620
lavadero          44034
balcon            42288
propiedades       41507
casa              40537
parrilla          33381
piscina           32148
luminoso          31865
suite             27102
placard           26145
terraza           25101
toilette          22841
subte             21984
linea             21824
patio             21169
vestidor          18011
jardin            17173
quincho           12850
aire              12767
acondicionado     12763
inmobiliaria      12184
sum               11665
dependencias      11316
max               11260
re                11258
cordoba           11195
amenities         10607
baulera           10512
estrenar          10383
vista             10309
de                 9768
san                9556
gimnasio           9363
lujoso             9251
plata              9215
del                9180
rosario            8607
villa              8473
dtype: int64

## Palabras para generar variables dummy
* garage
* lavadero
* balcon  
* casa
* parrilla
* piscina
* terraza
* patio
* jardin
* quincho
* aire acondicionado
* amenities
* estrenar
* gimnasio


Opcionales:
* baulera
* suite
* subte - linea
* vestidor


## Palabras para completar datos faltantes
* venta
* departamento

In [7]:
import unicodedata

pattern_desc = r'(\w\w\w\w\w*)'
pattern_desc_regex = re.compile(pattern_desc, flags=re.IGNORECASE )
result_match = data['description'].apply(lambda x: re.finditer(pattern_desc_regex, str(x).lower()))

list_words_desc = []

def generate_words_series_desc(matches):
    for matchNum, match in enumerate(matches, start=1):
        for groupNum in range(0, len(match.groups())):
            groupNum = groupNum + 1
            
            cadena = unicodedata.normalize('NFKD', match.group(groupNum)).encode('ASCII', 'ignore') 
            
            list_words_desc.append(cadena)
    return

result =  result_match.apply(generate_words_series_desc)

serie_desc = pd.Series(list_words_desc)
key_words_desc = serie_desc.value_counts()

In [8]:
display(key_words_desc.head(40))

b'cocina'          110088
b'bano'             98520
b'comedor'          96205
b'para'             85190
b'living'           70516
b'dormitorios'      67240
b'ambientes'        56062
b'dormitorio'       53025
b'piso'             50769
b'completo'         49287
b'departamento'     48517
b'balcon'           47086
b'pisos'            45367
b'planta'           44549
b'frente'           43095
b'casa'             39426
b'lavadero'         39128
b'excelente'        37242
b'amplio'           36923
b'edificio'         36296
b'venta'            35989
b'parrilla'         33136
b'cochera'          31810
b'cuenta'           31541
b'placard'          31247
b'suite'            31146
b'sobre'            28899
b'terraza'          28332
b'patio'            25833
b'vista'            24966
b'mesada'           24908
b'gran'             24171
b'propiedad'        23457
b'baja'             23351
b'ubicado'          22884
b'barrio'           22826
b'esta'             22535
b'zona'             22516
b'servicio' 

## Palabras para generar variables dummy
* balcon
* frente
* parrilla
* cochera
* terraza


Opcionales:
suite

## Palabras para completar datos faltantes
* dormitorio/s
* ambientes
* piso - planta
* departamento - casa - edificio
* venta
* ubicado - barrio

In [9]:
import unicodedata

pattern_m2 = r'(?P<before>(\w*\W?\s\W?){0,4})\s?(?P<value>\d{1,5})\s*m2'

pattern_m2_regex = re.compile(pattern_m2, flags=re.IGNORECASE )
result_match = data['description'].apply(lambda x: re.finditer(pattern_m2_regex, str(x).lower()))

list_words_desc_before = []
list_words_desc_values = []
list_words_desc_index = []
data_index = 0
df_m2 = pd.DataFrame(columns=('before', 'm2', 'index'))

def generate_m2_df(matches):
    global data_index
    for matchNum, match in enumerate(matches, start=1):  
        before = unicodedata.normalize('NFKD', match.group('before')).encode('utf-8', 'ignore').decode("utf-8") if match.group('before') != None else None
        value = match.group('value').strip()    
        
        list_words_desc_before.append(before)
        list_words_desc_values.append(value)
        list_words_desc_index.append(data_index)
    data_index += 1
    return

#result = result_match[0:5000].apply(generate_m2_df)
result = result_match.apply(generate_m2_df)

df_m2['m2'] = list_words_desc_values
df_m2['before'] = list_words_desc_before
df_m2['index'] = list_words_desc_index
display(df_m2.head(10))
print('cantidad de matches: ', df_m2.size)

,before,m2,index
0,cuadras del centro universitario.,106,6
1,totales:,78,6
2,cubiertos y,28,6
3,espejo de agua de,40,18
4,parrilla. amplio jardín de,1000,22
5,en 906 lotes de,1350,22
6,entrega: abril 2017superficie total:,138,40
7,superficie cubierta:,74,40
8,cubierta: 14 m2superficie descubierta:,50,40
9,y cubierta del departamento:,30,45


cantidad de matches:  145344


In [10]:
keys_m2 = list_words_desc_before
list_keys_words_m2 = []
for key in keys_m2:
    list_key_words = key.replace('.', '').replace(',', '').replace(':', '').split(' ')
    for word in list_key_words:
        if (word != '') & (len(word) > 3):
            list_keys_words_m2.append(word)
        
list_keys_words_m2

serie_keys_words_m2 = pd.Series(list_keys_words_m2)
serie_keys_words_m2_counts = serie_keys_words_m2.value_counts()
serie_keys_words_m2_counts.head(20)

superficie      4181
lote            2912
cubierta        2329
total           2269
sobre           2132
ambientes       1931
terreno         1904
cubiertos       1600
terraza         1099
casa            1063
frente           930
departamento     811
piso             759
patio            674
planta           611
dormitorio       571
balcón          557
local            544
ambiente         514
plantas          490
dtype: int64

In [11]:
WORD = "total" #escribir palabra a buscar

key_word_mask = df_m2['before'].apply(lambda x: True if WORD in x else False)
print('cantidad de veces que', WORD.upper(), 'fué encontrada: ', key_word_mask.sum())

df_m2[key_word_mask].head(20)

cantidad de veces que TOTAL fué encontrada:  2846


,before,m2,index
1,totales:,78,6
6,entrega: abril 2017superficie total:,138,40
13,"75 ,total",35,48
23,orientación oeste superficie total:,63,85
49,- superficie total:,335,117
90,para habitar sup total:,55,185
91,orientación sur. sup total:,41,185
94,esta sin terminar) total :,240,203
97,total:,387,211
98,7total cubierto :,140,211


In [32]:
import unicodedata

def generate_context_df(word, column, rows=5000, has_context=True):
    global data_index
    if has_context is True:
        pattern_context = r'(?P<context>(\w*\W?\s\W?){0,4}' + word + '(\w*\W?\s\W?){0,4})'
    else:    
        pattern_context = word

    pattern_context_regex = re.compile(pattern_context, flags=re.IGNORECASE )
    result_match = data[column].apply(lambda x: re.finditer(pattern_context_regex, str(x).lower()))

    list_words_desc_context = []
    list_words_desc_index = []
    data_index = 0
    df_context = pd.DataFrame()
        
    def generate_context_df(matches):
        global data_index
        for matchNum, match in enumerate(matches, start=1):  
            if has_context is True:
                context = match.group('context')     
                list_words_desc_context.append(context)
            list_words_desc_index.append(data_index)
        data_index += 1
        return
    
    if rows == "ALL":
        result = result_match.apply(generate_context_df)
    else:
        result = result_match[0:rows].apply(generate_context_df)
        
    if has_context is True:
        df_context['context'] = list_words_desc_context
        
    df_context['index'] = list_words_desc_index
    print('cantidad de matches: ', df_context.shape[0], ' en ', rows, ' registros buscados')
    return df_context

In [34]:
generate_context_df("expensas", "description", 5000, True).head(20)

cantidad de matches:  506  en  5000  registros buscados


,context,index
0,"dormitorio con placard. bajas expensas, excele...",2
1,entrada independiente - sin expensas! todo en ...,6
2,los servicios . bajas expensas,23
3,"con reubicación, no paga expensas, aguas indep...",34
4,al igual que las expensas que pueden estar,35
5,seguridad 24 hs. bajas expensas. excelente acc...,43
6,expensas aproximadas $,50
7,baulera independiente en sótano. expensas $,52
8,muy buena renta bajas expensas valor de venta,60
9,070. expensas $,74
